# `chart_ipynb`  Time Series  

`chart_ipynb` provides additional functions specifically for time series data. Instead of using the function directly, we first set up the data and options manually to show how it works. 

In `time_series`, we use [`pandas_datareader`](https://pandas-datareader.readthedocs.io/en/latest/) to read stock data from [`quandl`](https://pandas-datareader.readthedocs.io/en/latest/readers/quandl.html) by default. Other data websites can also be used to access different kinds of data ([More details can be found here](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html)). 

In [12]:
from chart_ipynb import utils
from chart_ipynb.chart_framework import ChartSuperClass
import numpy as np
import pandas as pd
import pandas_datareader
import pandas_datareader.data as web
import datetime
import time

For free multiple access to data, `quandl` requires api_key by creating your own accounts on the [official website](https://www.quandl.com/). API key can be found under your Account Settings.  

In the following examples, we will use data of Apple, Amazon, Google. 

In [13]:
api_key = '1JFowowyzc-FnajAsDkY'
start = datetime.datetime(2017,1,1)
end = datetime.datetime(2018,1,1)
aapl = web.DataReader('AAPL',"quandl", start, end, api_key = api_key)
amzn = web.DataReader('AMZN',"quandl", start, end, api_key = api_key)
googl = web.DataReader('GOOGL',"quandl", start, end, api_key = api_key)

In [3]:
aapl.head()

In [4]:
amzn.head()

In [5]:
googl.head()

We need to format the data to the structure available to feed into the chart initialization function.  

The following function return two values, a list of price values, a list of date string.

In [7]:
def data_format(dataset, val_col):
    """
    dataset: pd.DataFrame
    val_col: the column name for the target value. e.g 'Close'
    """
    data = dataset[val_col]
    idx_reset_df = dataset.reset_index()
    if 'Date' not in idx_reset_df.columns:
        return 'please rename the date columns to "Date"'
    sort_df = idx_reset_df.sort_values(by='Date')
    sort_df['Date']=sort_df['Date'].astype(str)
    return  list(sort_df[val_col]), list(sort_df['Date'])

In [8]:
aapl_val, aapl_label = data_format(aapl, 'Close')
amzn_val, amzn_label = data_format(amzn, 'Close')
googl_val, googl_label = data_format(googl, 'Close')

`utils` provides helper functions to create dataset formats and data formats.  
The `label` in the dataset is the ticker symbol of the company, while `labels` in the data is a list of Date string which will be present as x axis.

In [9]:
dataset1 = utils.dataset(
                label = 'AAPL',
                backgroundColor = utils.color_rgb('red',0.5),
                borderColor = utils.color_rgb('red'),
                data = aapl_val,
                type = 'line',
                pointRadius = 0,
                fill = False,
                lineTension = 0,
                borderWidth = 2
            )
dataset2 = utils.dataset(
                label = 'AMZN',
                backgroundColor = utils.color_rgb('blue',0.5),
                borderColor = utils.color_rgb('blue'),
                data = amzn_val,
                type = 'line',
                pointRadius = 0,
                fill = False,
                lineTension = 0,
                borderWidth = 2
            )
dataset3 = utils.dataset(
                label = 'GOOGL',
                backgroundColor = utils.color_rgb('green',0.5),
                borderColor = utils.color_rgb('green'),
                data = googl_val,
                type = 'line',
                pointRadius = 0,
                fill = False,
                lineTension = 0,
                borderWidth = 2
            )
data = utils.data(
    labels = aapl_label,
    datasets = [dataset1,dataset2,dataset3]
)

The configuration is required to initialize the chart. It contains type, data, options.   
The value of type will be a string indicating the type of chart, such as 'line', 'bar', and 'bubble' etc. The dictionary of options contains many more features including title, legend scales and elements.

In [10]:
config = utils.config(
        type = 'line',
        data = data,
        options = utils.options(
            animation = {
                'duration': 0
            },
            scales = {
                'xAxes': [{
                    'display':True,
                    'scaleLabel':{
                        'display':True,
                        'labelString':'Date'
                    }
                    ,'ticks': {
                        'major': {
                            'enabled': True,
                            'fontStyle': 'bold'
                        },
                        'source': 'data',
                        'autoSkip': True,
                        'autoSkipPadding': 10,
                        'maxRotation': 60,
                    },
                }],
                'yAxes': [{
                    'gridLines': {
                        'drawBorder': False
                    },
                    'scaleLabel': {
                        'display': True,
                        'labelString': 'Closing price ($)'
                    }
                }]
            },
        )
)


After setting up the configuration, we are ready to initialize the line chart by creating a ChartSuperClass object which is the super class for all the charts in `chart_ipynb`.

In [11]:
line_chart = ChartSuperClass()
line_chart.initialize_chart(width=800, config=config)
line_chart

ChartSuperClass(status='deferring flush until render')

##  `time_series` line chart

Now, we can directly use the function called `time_series_Chart` provided in `time_series` to create line chart.


`time_series_Chart` support two types of charts: line and bar. 

```
time_series_Chart(_chart_type, ticker_symbol, val_col, date_col = None, 
                  start=None, end=None, 
                  data_provide = False, input_dataset = None,
                  website = None, api_key = None, 
                  multi_axis = False, axis_label = None, stacked = False,
                  options = None, xAxes = None, yAxes = None,
                  colors=None, backgroundColor = None, borderColor = None, 
                  title = None,
                  fill = False,
                  width=800,
                  **other_arguments
                  )
```

- `_chart_type`: the type of chart, 'line' or 'bar     
- `ticker_symbol`: if use inner stock dataset, it will be ticker symbol of company; if self provide data, it will be the name of datasets shown in the legend
- `val_col`: the name of value column  
- `date_col`: the name of date column  
- `start`: start date; a string format in  'yyyy-m-d'  
- `end`: end date; a string format in  'yyyy-m-d'   
- `data_provide`: self provide data or not; default is False    
- `input_dataset`: if data_provide=True, must provide your own data   
- `website`: the website you want to access the data   
- `api_key`: API key to access the data from the website   
- `multi_axis`: only work for two datasets    
- `axis_label`: axis label 
- `stacked`: only work for bar chart and multi_aixs = False    
- More arguments refer to [here](https://github.com/AaronWatters/Chart_ipynb/blob/master/chart_ipynb/time_series.py)

In [1]:
from chart_ipynb import time_series 

## Stock Closing prices from quandl  

### Line Chart

In [3]:
start = '2017-1-1'
end = '2018-1-1'
symbols = ['AAPL','AMZN','GOOGL']
colors = ['red', 'blue', 'green']
col = 'Close'
time_series.time_series_Chart('line', symbols, col, start = start, end = end, colors = colors,
                              website='quandl', title='Closing Price ($)')

Line(status='deferring flush until render')

### Stacked Bar Chart

In [6]:
time_series.time_series_Chart('bar', symbols, col, start = start, end = end, colors = ['violet', 'midnightblue', 'cyan'], 
                              website='quandl',stacked=True, title='Closing Price ($)')

Bar(status='deferring flush until render')

## Stock Open prices from quandl - Multi axis

In [10]:
start = '2017-1-1'
end = '2018-1-1'
symbols = ['AAPL','AMZN']
colors = ['purple', 'brown']
val_col = 'Open'
time_series.time_series_Chart('line', symbols, val_col, start = start, end = end, 
                              website='quandl', multi_axis = True,
                              colors = colors,
                              title = 'Opening Price ($)')

Line(status='deferring flush until render')

## Self Provide Datasets 

The following example show how the function works when using your own datasets.

### Line Chart

In [16]:
symbols = ['AAPL','AMZN', 'GOOGL']
input_dataset = [aapl, amzn, googl]
colors = ['salmon','seagreen','royalblue']
val_col = 'Close'
date_col = 'Date'
time_series.time_series_Chart('line', symbols, val_col, 
                           date_col = date_col,
                           start = start, end = end, 
                           data_provide=True, 
                           input_dataset = input_dataset,
                           colors = colors,
                           title = "Closing Price ($)")

Line(status='deferring flush until render')

### Bar Chart

In [17]:
colors = ['salmon','seagreen','royalblue']
time_series.time_series_Chart('bar', symbols, val_col, 
                           date_col = date_col,
                           start = start, end = end, 
                           data_provide=True, 
                           input_dataset = input_dataset,
                           stacked=True,
                           colors = colors,
                           title = "Closing Price ($)")

Bar(status='deferring flush until render')